In [7]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv('/kaggle/input/diffusion-generated-data/generated_dataset.csv')
X = df.drop(columns=['Label']).values  # All features except the target
y = df['Label'].values  # Target: 1 for malicious, 0 for benign

# Train-Test Split (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling: Normalize the data (standardize)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [8]:
import torch.nn as nn
import torch.optim as optim

class MaliciousTrafficModel(nn.Module):
    def __init__(self, input_dim):
        super(MaliciousTrafficModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)  # Increased number of neurons
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

# Define the model and move it to GPU if available
input_dim = X_train_scaled.shape[1]  # Number of input features
model = MaliciousTrafficModel(input_dim)

# Choose a loss function and optimizer
loss_function = nn.BCELoss()  # Binary Cross-Entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)


In [9]:
from tqdm import tqdm
import matplotlib.pyplot as plt

# Lists to store accuracy and loss values for plotting
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

num_epochs = 50  

# tqdm progress bar for epochs
for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
    print(f"Epoch {epoch+1}\n-------------------------------")
    
    # Training phase
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs).squeeze()  # Forward pass
        loss = loss_function(outputs, labels.float())  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()

        # Predictions and accuracy calculation
        predicted = (outputs > 0.5).long()
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    train_accuracy = correct_predictions / total_predictions
    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(train_accuracy)

    # Validation/Test phase
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs).squeeze()
            loss = loss_function(outputs, labels.float())
            test_loss += loss.item()

            predicted = (outputs > 0.5).long()
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    test_accuracy = correct_predictions / total_predictions
    test_losses.append(test_loss / len(test_loader))
    test_accuracies.append(test_accuracy)

    print(
        f"Train Accuracy: {train_accuracy*100:.2f}%  Test Accuracy: {test_accuracy*100:.2f}%  "
        f"Train Loss: {train_losses[-1]:.4f}  Test Loss: {test_losses[-1]:.4f}"
    )


Training Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1
-------------------------------
Train Accuracy: 56.74%  Test Accuracy: 56.17%  Train Loss: 0.6845  Test Loss: 0.6870


Training Epochs:   2%|▏         | 1/50 [00:10<08:51, 10.85s/it]

Epoch 2
-------------------------------
Train Accuracy: 56.77%  Test Accuracy: 56.17%  Train Loss: 0.6841  Test Loss: 0.6857


Training Epochs:   4%|▍         | 2/50 [00:21<08:35, 10.74s/it]

...

Training Epochs:  96%|█████████▌| 48/50 [09:59<00:24, 12.41s/it]

Epoch 49
-------------------------------
Train Accuracy: 56.77%  Test Accuracy: 56.17%  Train Loss: 0.6839  Test Loss: 0.6863


Training Epochs:  98%|█████████▊| 49/50 [10:11<00:12, 12.34s/it]

Epoch 50
-------------------------------
Train Accuracy: 56.77%  Test Accuracy: 56.17%  Train Loss: 0.6839  Test Loss: 0.6864


Training Epochs: 100%|██████████| 50/50 [10:23<00:00, 12.47s/it]


In [16]:
import torch

# Script the model
scripted_model = torch.jit.script(model)

# Save the scripted model
scripted_model_path = "trained_model.pt"
scripted_model.save(scripted_model_path)

print(f"Scripted model saved to {scripted_model_path}")

Scripted model saved to trained_model.pt


In [18]:
def save_row_as_json(df, row_index, output_path):
    row_features = df.iloc[row_index].drop(labels=['Label']).to_dict()  # Drop the target column ('Label') and convert to dict
    with open(output_path, 'w') as json_file:
        json.dump(row_features, json_file, indent=4)
    print(f"Row {row_index} saved as JSON to {output_path}")

In [20]:
import json
save_row_as_json(df, 1, 'ligne_1_features.json')

Row 1 saved as JSON to ligne_1_features.json
